In [6]:
import pandas as pd
import numpy as np
from catboost import Pool, CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit 
import optuna 
import json
from sklearn.metrics import mean_absolute_error

In [7]:
#Load inn datasets
X_test  = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/X_test.parquet')
X_train = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/X_train.parquet')
y_train = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train.parquet')
y_train_a = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train_a.parquet')
y_train_b = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train_b.parquet')
y_train_c = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train_c.parquet')

In [8]:
def test_train_split(df):
    dates_2 = (df.index >= '2023-05-01') & (df.index <= '2023-06-15')
    dates_1 = (df.index >= '2020-05-01') & (df.index <= '2020-06-25')

    test_set = df[dates_1 | dates_2]

    training_set = df[~(dates_1 | dates_2)]

    X_train = training_set.drop("pv_measurement", axis=1)
    y_train = training_set['pv_measurement']

    X_test = test_set.drop("pv_measurement", axis=1)
    y_test = test_set['pv_measurement'] 

    
    
    return X_train, X_test, y_train, y_test


X_train_new_c, X_test_new_c, y_train_new_c, y_test_c = test_train_split(pd.concat([X_train[X_train["location"] == "C"].drop("location", axis=1), y_train_c], axis=1))

In [9]:
#Create a pool of data

train_pool_c = Pool(X_train_new_c, y_train_new_c)
test_pool_c = Pool(X_test_new_c) 

In [11]:
#For location A
def objective(trial, X_train, y_train):
    params = {
        "iterations": trial.suggest_int("iterations", 300, 3000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 13),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 2, 10),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.3, 1.0),
        "random_strength": trial.suggest_float("random_strength", 0.3, 1.0),
        "border_count": trial.suggest_int("border_count", 1, 1000),
        "rsm": trial.suggest_float("rsm", 0.05, 1),
    }

    catboost_model_c = CatBoostRegressor(**params, verbose=100)
    catboost_model_c.fit(train_pool_c)
    pred_c = pd.DataFrame(catboost_model_c.predict(test_pool_c))
    MAE_c = mean_absolute_error(y_test_c, pred_c)
    return MAE_c

study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=80)

[I 2023-11-07 08:22:43,666] A new study created in memory with name: no-name-bc4633e0-9098-49f1-b819-3074929938af


0:	learn: 148.3129687	total: 52.2ms	remaining: 1m 4s
100:	learn: 62.7742200	total: 215ms	remaining: 2.43s
200:	learn: 61.1178012	total: 374ms	remaining: 1.94s
300:	learn: 60.4596593	total: 539ms	remaining: 1.68s
400:	learn: 60.0060718	total: 693ms	remaining: 1.45s
500:	learn: 59.6728801	total: 870ms	remaining: 1.29s
600:	learn: 59.4240480	total: 1.04s	remaining: 1.11s
700:	learn: 59.2308958	total: 1.2s	remaining: 926ms
800:	learn: 59.0696748	total: 1.36s	remaining: 751ms
900:	learn: 58.9359081	total: 1.52s	remaining: 576ms
1000:	learn: 58.8175569	total: 1.68s	remaining: 404ms
1100:	learn: 58.7105760	total: 1.86s	remaining: 238ms


[I 2023-11-07 08:22:45,914] Trial 0 finished with value: 64.3517265556924 and parameters: {'iterations': 1242, 'learning_rate': 0.06201532080191678, 'depth': 1, 'min_data_in_leaf': 81, 'l2_leaf_reg': 5, 'bagging_temperature': 0.5635146379824496, 'random_strength': 0.745706042230049, 'border_count': 213, 'rsm': 0.3139915355130493}. Best is trial 0 with value: 64.3517265556924.


1200:	learn: 58.6162738	total: 2.02s	remaining: 68.9ms
1241:	learn: 58.5797943	total: 2.09s	remaining: 0us
0:	learn: 147.5637302	total: 2.07ms	remaining: 4.39s
100:	learn: 61.3542992	total: 225ms	remaining: 4.49s
200:	learn: 59.8816541	total: 454ms	remaining: 4.33s
300:	learn: 59.2658054	total: 678ms	remaining: 4.09s
400:	learn: 58.8943948	total: 905ms	remaining: 3.87s
500:	learn: 58.6418225	total: 1.14s	remaining: 3.67s
600:	learn: 58.4566293	total: 1.37s	remaining: 3.46s
700:	learn: 58.3126311	total: 1.61s	remaining: 3.25s
800:	learn: 58.1944180	total: 1.84s	remaining: 3.03s
900:	learn: 58.0941441	total: 2.06s	remaining: 2.78s
1000:	learn: 58.0065959	total: 2.29s	remaining: 2.56s
1100:	learn: 57.9296966	total: 2.55s	remaining: 2.35s
1200:	learn: 57.8603731	total: 2.82s	remaining: 2.15s
1300:	learn: 57.7960537	total: 3.07s	remaining: 1.93s
1400:	learn: 57.7362309	total: 3.33s	remaining: 1.71s
1500:	learn: 57.6811730	total: 3.58s	remaining: 1.47s
1600:	learn: 57.6295153	total: 3.81s	re

[I 2023-11-07 08:22:51,212] Trial 1 finished with value: 63.25269725360107 and parameters: {'iterations': 2118, 'learning_rate': 0.06157354611355663, 'depth': 1, 'min_data_in_leaf': 38, 'l2_leaf_reg': 2, 'bagging_temperature': 0.8748776332496979, 'random_strength': 0.35634717380032965, 'border_count': 737, 'rsm': 0.8918009107524033}. Best is trial 1 with value: 63.25269725360107.


...

0:	learn: 150.1627794	total: 33.5ms	remaining: 1m 12s
100:	learn: 55.0295342	total: 4.32s	remaining: 1m 28s
200:	learn: 49.2453121	total: 8.74s	remaining: 1m 25s
300:	learn: 47.2922635	total: 13s	remaining: 1m 20s
400:	learn: 45.5105172	total: 16.9s	remaining: 1m 14s
500:	learn: 43.9141614	total: 21s	remaining: 1m 9s
600:	learn: 42.4754325	total: 25.1s	remaining: 1m 5s
700:	learn: 41.2431539	total: 29.5s	remaining: 1m 1s
800:	learn: 40.0958164	total: 33.9s	remaining: 57.7s
900:	learn: 39.0626719	total: 38s	remaining: 53.3s
1000:	learn: 37.9703678	total: 42.6s	remaining: 49.5s
1100:	learn: 37.0386034	total: 46.9s	remaining: 45.4s
1200:	learn: 36.2035145	total: 51.2s	remaining: 41.1s
1300:	learn: 35.3395791	total: 55.4s	remaining: 36.8s
1400:	learn: 34.5877760	total: 59.5s	remaining: 32.5s
1500:	learn: 33.8451065	total: 1m 3s	remaining: 28.3s
1600:	learn: 33.1753636	total: 1m 8s	remaining: 24.1s
1700:	learn: 32.4650722	total: 1m 13s	remaining: 19.9s
1800:	learn: 31.7700993	total: 1m 17s	

[I 2023-11-07 10:29:27,870] Trial 78 finished with value: 57.02321736650033 and parameters: {'iterations': 2165, 'learning_rate': 0.023622966536140023, 'depth': 7, 'min_data_in_leaf': 18, 'l2_leaf_reg': 7, 'bagging_temperature': 0.6094991472719161, 'random_strength': 0.5048869371272217, 'border_count': 819, 'rsm': 0.5908931457712421}. Best is trial 48 with value: 56.710142980507335.


2164:	learn: 29.6980855	total: 1m 32s	remaining: 0us
0:	learn: 149.7976201	total: 42.6ms	remaining: 1m 36s
100:	learn: 52.6704859	total: 4.15s	remaining: 1m 28s
200:	learn: 47.6791159	total: 8.27s	remaining: 1m 24s
300:	learn: 45.4125970	total: 12.5s	remaining: 1m 20s
400:	learn: 43.4091281	total: 17s	remaining: 1m 18s
500:	learn: 41.4822868	total: 21.4s	remaining: 1m 14s
600:	learn: 39.5369573	total: 25.9s	remaining: 1m 11s
700:	learn: 37.9053698	total: 30.1s	remaining: 1m 6s
800:	learn: 36.4524048	total: 35.1s	remaining: 1m 3s
900:	learn: 35.1544246	total: 39.7s	remaining: 59.7s
1000:	learn: 33.9227160	total: 44.3s	remaining: 55.5s
1100:	learn: 32.7394165	total: 48.7s	remaining: 51.1s
1200:	learn: 31.7437350	total: 53.1s	remaining: 46.6s
1300:	learn: 30.7888165	total: 57.4s	remaining: 42.1s
1400:	learn: 29.8464523	total: 1m 2s	remaining: 37.8s
1500:	learn: 29.0870900	total: 1m 5s	remaining: 33.1s
1600:	learn: 28.2817479	total: 1m 9s	remaining: 28.6s
1700:	learn: 27.5053135	total: 1m 

[I 2023-11-07 10:31:06,580] Trial 79 finished with value: 57.66030691602226 and parameters: {'iterations': 2255, 'learning_rate': 0.026814697635350382, 'depth': 8, 'min_data_in_leaf': 15, 'l2_leaf_reg': 8, 'bagging_temperature': 0.6383463899813797, 'random_strength': 0.6214086412652821, 'border_count': 304, 'rsm': 0.5683844039902519}. Best is trial 48 with value: 56.710142980507335.


2254:	learn: 23.8089052	total: 1m 37s	remaining: 0us


In [7]:
#to output the best paramaters
print(study.best_params)

#to output the best score returned from the trials
print(study.best_value)

with open("optuna-best-parameters_c.txt", "w") as file:
    file.write("Best paramaters: \n")
    file.write(json.dumps(study.best_params))  # Write the first string followed by a newline character
    file.write("\n")
    file.write("best score MAE: \n")
    file.write(json.dumps(study.best_value))  # Write the second string followed by a newline character

{'iterations': 2004, 'learning_rate': 0.016429110886618033, 'depth': 8, 'colsample_bylevel': 0.7338575760364532, 'min_data_in_leaf': 90, 'l2_leaf_reg': 5, 'has-time': False, 'bagging_temperature': 0.6414768311294344, 'random_strength': 0.4206698081688342, 'border_count': 487, 'rsm': 0.012360675671015169, 'nan_mode': 'min'}
57.13529571086957
